In [1]:
from param import ml_names, pers_names, data_name, data_path, results_path, thres_sel_feat
from ml_models import loocv_day, agg_feat_imp
from ml_models import MAE, MSE, RMSE, MAPE, R2
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(results_path)
df_ml_input_new = pd.read_csv('ml_input.csv', index_col = 0)

In [ ]:
ml_model_name = 'rf'
ls_results_rf = list()
ls_feats_rf = list()
for pers_name in pers_names:
    ls_results_loocv_rf, ls_feats_loocv_rf = loocv_day(df_ml_input_new, pers_name, ml_model_name)
    ls_results_rf.extend(ls_results_loocv_rf)
    ls_feats_rf.extend(ls_feats_loocv_rf)
    df_results_rf_raw = pd.DataFrame(ls_results_loocv_rf, columns = ['Personality', 'Participant ID', 'Y test', 'Y pred', 'Best param'])
    df_feats_rf_raw = pd.DataFrame()
    df_feats_rf_raw['Personality'] = df_results_rf_raw['Personality'].tolist()
    df_feats_rf_raw['Participant ID'] = df_results_rf_raw['Participant ID'].tolist()
    df_feats_rf_raw['Feats'] = ls_feats_loocv_rf
    df_results_rf_raw.to_csv('rf_' + pers_name + '_results_raw.csv', index=None)
    df_feats_rf_raw.to_csv('rf_' + pers_name + '_feats_raw.csv', index=None)

In [ ]:
df_results_rf = pd.DataFrame(ls_results_rf, columns = ['Personality', 'Participant ID', 'MI_iteration', 'Y test', 'Y pred'])
df_results_rf.head()

In [ ]:
ls_evalutation_rf = list()
df_results_rf_groups = df_results_rf.groupby('Personality')
df_results_rf_group_names = df_results_rf_groups.size().index
for df_results_rf_group_name in df_results_rf_group_names:
    ls_evaluation_row_rf = [df_results_rf_group_name]
    df_results_rf_group = df_results_rf_groups.get_group(df_results_rf_group_name)
    ls_test_vals_rf = list()
    ls_pred_vals_rf = list()
    for val in df_results_rf_group['Y test'].tolist():
        if len(val) < 1:
            print('Error')
        
        ls_test_vals_rf.extend(val)

    for val in df_results_rf_group['Y pred'].tolist():
        if len(val) < 1:
            print('Error')
            
        ls_pred_vals_rf.extend(val)
    
    df_evaluation_input_rf = pd.DataFrame()
    df_evaluation_input_rf['Y_test'] = ls_test_vals_rf
    df_evaluation_input_rf['Y_pred'] = ls_pred_vals_rf
    
#     df_evalution_output_xgb = evaluate(df_evaluation_input_xgb)
    ls_evalutation_rf.append([df_results_rf_group_name, 
                               MAE(ls_pred_vals_rf, ls_test_vals_rf),
                               MSE(ls_pred_vals_rf, ls_test_vals_rf),
                               RMSE(ls_pred_vals_rf, ls_test_vals_rf),
                               MAPE(ls_pred_vals_rf, ls_test_vals_rf),
                               R2(ls_pred_vals_rf, ls_test_vals_rf)])

In [ ]:
df_results_summary = pd.DataFrame(ls_evalutation_rf, columns = ['Personality', 'MAE', 'MSE', 'RMSE', 'MAPE(%)', 'R2'])
df_results_summary.to_csv('rf_results_summary.csv', index=None)

In [ ]:
dict_feats_summary_imp_rf, dict_feats_summary_freq_rf = agg_feat_imp(ls_feats_rf)

In [ ]:
ls_feats_summary_imp_rf = list()
for feat_name in dict_feats_summary_imp_rf:
    ls_row = [feat_name, dict_feats_summary_imp_rf[feat_name]]
    ls_feats_summary_imp_rf.append(ls_row)

df_feats_summary_imp_rf = pd.DataFrame(ls_feats_summary_imp_rf, columns = ['Feature', 'Importance'])
df_feats_summary_imp_rf.to_csv('rf_feats_imp_summary.csv',index=None)

In [ ]:
ls_feats_summary_freq_rf = list()
for feat_name in dict_feats_summary_freq_rf:
    ls_row = [feat_name, dict_feats_summary_freq_rf[feat_name]]
    ls_feats_summary_freq_rf.append(ls_row)

df_feats_summary_freq_rf = pd.DataFrame(ls_feats_summary_freq_rf, columns = ['Feature', 'Frequency'])
df_feats_summary_freq_rf.to_csv('rf_feats_freq_summary.csv',index=None)